## Testing Yu-Gi-Oh API card grabbing

### Import required packages

In [1]:
import numpy as np
import pandas as pd
import requests
from pandas.io.json import json_normalize

print("All libraries imported successfully!")

# API source: https://db.ygoprodeck.com/api-guide/

All libraries imported successfully!


In [2]:
# function to add parameters to the API call
def add_parameter(f, v, p):
    if p == "?":
        return "{}{}={}".format(p,f,v)
    else:
        return "&{}{}={}".format(p,f,v)

In [3]:
# function that extracts the category of the venue
def separate_prices(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [5]:
params = "?"
url = "https://db.ygoprodeck.com/api/v7/cardinfo.php"

#params = add_parameter("name","Dark Magician",params)
params = add_parameter("archetype","Prank-Kids",params)

if params != "?":
    url = url + params
results = requests.get(url).json()
results
print(url)

https://db.ygoprodeck.com/api/v7/cardinfo.php?archetype=Prank-Kids


In [76]:
cards = results['data']
cards_db = json_normalize(cards)
pd.set_option('display.max_colwidth', 60)

# Use below to check table
#cards_db

C:\Users\dsalt\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [77]:
# Handle card prices and turn into sepearate columns

prices = json_normalize(data = cards, record_path = 'card_prices', meta = ['name'])
cards_db = pd.merge(cards_db, prices)

cards_db.drop(columns='card_prices',inplace=True)
#cards_db['cardmarket_price'] = cards_db['cardmarket_price'].apply(lambda x: x*1.1)  For converting to USD later
cards_db

C:\Users\dsalt\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,type,desc,atk,def,level,race,attribute,archetype,card_sets,card_images,linkval,linkmarkers,cardmarket_price,tcgplayer_price,ebay_price,amazon_price,coolstuffinc_price
0,81997228,Prank-Kids Battle Butler,Fusion Monster,"""Prank-Kids Lampsies"" + ""Prank-Kids Dropsies"" + ""Prank-K...",3000.0,3000.0,10.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': 'HISU-EN01...","[{'id': 81997228, 'image_url': 'https://storage.googleap...",NaN,NaN,2.62,4.06,3.99,4.50,3.99
1,43886072,Prank-Kids Bow-Wow-Bark,Link Monster,"2 ""Prank-Kids"" monsters\r\nA ""Prank-Kids"" monster this c...",2000.0,NaN,NaN,Pyro,FIRE,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': 'HISU-EN02...","[{'id': 43886072, 'image_url': 'https://storage.googleap...",2.0,"[Right, Bottom]",0.25,1.22,1.50,0.99,1.49
2,17382973,Prank-Kids Dodo-Doodle-Doo,Link Monster,"2 ""Prank-Kids"" monsters\r\nIf this card is Link Summoned...",2000.0,NaN,NaN,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': 'HISU-EN02...","[{'id': 17382973, 'image_url': 'https://storage.googleap...",2.0,"[Left, Bottom]",7.92,9.04,5.99,10.50,5.99
3,55725117,Prank-Kids Dropsies,Effect Monster,If this card is sent to the GY as material for the Fusio...,1000.0,1000.0,2.0,Aqua,WATER,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': 'HISU-EN01...","[{'id': 55725117, 'image_url': 'https://storage.googleap...",NaN,NaN,0.47,1.09,1.09,2.01,0.99
4,81119816,Prank-Kids Fansies,Effect Monster,If this card is sent to the GY as material for the Fusio...,0.0,2000.0,1.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': 'HISU-EN01...","[{'id': 81119816, 'image_url': 'https://storage.googleap...",NaN,NaN,3.08,1.79,0.99,2.20,2.99
5,18236002,Prank-Kids Lampsies,Effect Monster,If this card is sent to the GY as material for the Fusio...,1500.0,500.0,3.0,Pyro,FIRE,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': 'HISU-EN01...","[{'id': 18236002, 'image_url': 'https://storage.googleap...",NaN,NaN,3.13,1.46,27.00,0.84,0.99
6,25725326,Prank-Kids Meow-Meow-Mu,Link Monster,"1 Level 4 or lower ""Prank-Kids"" monster\r\nYou can only ...",1000.0,NaN,NaN,Rock,EARTH,Prank-Kids,"[{'set_name': 'Phantom Rage', 'set_code': 'PHRA-EN049', ...","[{'id': 25725326, 'image_url': 'https://storage.googleap...",1.0,[Bottom],0.48,0.45,0.99,0.89,1.99
7,79059098,Prank-Kids Pandemonium,Spell Card,"During the Main Phase: Fusion Summon 1 ""Prank-Kids"" Fusi...",NaN,NaN,NaN,Quick-Play,NaN,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': 'HISU-EN02...","[{'id': 79059098, 'image_url': 'https://storage.googleap...",NaN,NaN,0.42,1.07,27.00,1.78,0.99
8,16269385,Prank-Kids Place,Spell Card,"When this card is activated: You can add 1 ""Prank-Kids"" ...",NaN,NaN,NaN,Field,NaN,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': 'HISU-EN02...","[{'id': 16269385, 'image_url': 'https://storage.googleap...",NaN,NaN,23.62,18.18,27.00,17.50,15.99
9,15447747,Prank-Kids Plan,Trap Card,During the Main Phase: You can activate this effect; imm...,NaN,NaN,NaN,Continuous,NaN,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': 'HISU-EN02...","[{'id': 15447747, 'image_url': 'https://storage.googleap...",NaN,NaN,0.21,0.36,0.99,0.29,0.39


In [7]:
# Uncomment below line if full text should be displayed
#pd.set_option('display.max_colwidth', -1)
cards_db.head()

,id,name,type,desc,atk,def,level,race,attribute,archetype,card_sets,card_images,linkval,linkmarkers,cardmarket_price,tcgplayer_price,ebay_price,amazon_price,coolstuffinc_price
0,81997228,Prank-Kids Battle Butler,Fusion Monster,"""Prank-Kids Lampsies"" + ""Prank-Kids Dropsies"" ...",3000.0,3000.0,10.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 81997228, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
1,43886072,Prank-Kids Bow-Wow-Bark,Link Monster,"2 ""Prank-Kids"" monsters\r\nA ""Prank-Kids"" mons...",2000.0,NaN,NaN,Pyro,FIRE,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 43886072, 'image_url': 'https://storag...",2.0,"[Right, Bottom]",0.68,0.38,4.25,0.62,0.39
2,17382973,Prank-Kids Dodo-Doodle-Doo,Link Monster,"2 ""Prank-Kids"" monsters\r\nIf this card is Lin...",2000.0,NaN,NaN,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 17382973, 'image_url': 'https://storag...",2.0,"[Left, Bottom]",0.68,0.38,4.25,0.62,0.39
3,55725117,Prank-Kids Dropsies,Effect Monster,If this card is sent to the GY as material for...,1000.0,1000.0,2.0,Aqua,WATER,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 55725117, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
4,81119816,Prank-Kids Fansies,Effect Monster,If this card is sent to the GY as material for...,0.0,2000.0,1.0,Thunder,WIND,Prank-Kids,"[{'set_name': 'Hidden Summoners', 'set_code': ...","[{'id': 81119816, 'image_url': 'https://storag...",NaN,NaN,0.68,0.38,4.25,0.62,0.39
